# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846872671939                   -0.70    4.2
  2   -7.852328244476       -2.26       -1.53    1.0
  3   -7.852646044257       -3.50       -2.52    3.2
  4   -7.852646676034       -6.20       -3.34    2.0
  5   -7.852646685813       -8.01       -4.76    1.2
  6   -7.852646686726       -9.04       -5.21    4.2
  7   -7.852646686730      -11.47       -5.89    1.8
  8   -7.852646686730      -12.46       -7.51    1.8
  9   -7.852646686730      -14.45       -7.73    2.8
 10   -7.852646686730   +  -14.45       -8.79    2.5
 11   -7.852646686730   +  -15.05       -9.97    2.5
 12   -7.852646686730      -15.05      -11.02    2.8
 13   -7.852646686730      -14.75      -11.57    2.0
 14   -7.852646686730   +    -Inf      -12.14    1.8


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846771271043                   -0.70           3.8
  2   -7.852527905767       -2.24       -1.62   0.80    5.0
  3   -7.852608729628       -4.09       -2.80   0.80    5.2
  4   -7.852646565906       -4.42       -3.56   0.80    7.2
  5   -7.852646683306       -6.93       -4.44   0.80    8.8
  6   -7.852646686641       -8.48       -5.07   0.80   10.5
  7   -7.852646686725      -10.08       -6.30   0.80   11.8
  8   -7.852646686730      -11.30       -7.19   0.80   13.8
  9   -7.852646686730      -13.71       -7.67   0.80   15.8
 10   -7.852646686730   +  -15.05       -9.43   0.80   15.8
 11   -7.852646686730   +    -Inf       -9.16   0.80   19.0
 12   -7.852646686730   +    -Inf      -10.23   0.80   19.0
 13   -7.852646686730      -15.05      -11.39   0.80   20.5
 14   -7.852646686730      -14.57      -11.96   0.80   22.8
 15   -7.852646686730   +  -14.57      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.383586e+01     3.604874e+00
 * time: 0.5177390575408936
     1     1.329422e+00     2.232967e+00
 * time: 0.7850120067596436
     2    -1.655567e+00     2.702838e+00
 * time: 0.8137390613555908
     3    -3.728392e+00     2.024694e+00
 * time: 0.8556339740753174
     4    -5.110854e+00     1.955120e+00
 * time: 0.8968391418457031
     5    -6.768261e+00     1.012402e+00
 * time: 0.9383931159973145
     6    -7.434796e+00     6.678200e-01
 * time: 0.980828046798706
     7    -7.658545e+00     5.210874e-01
 * time: 1.0095620155334473
     8    -7.737768e+00     1.679822e-01
 * time: 1.0382981300354004
     9    -7.788356e+00     1.105200e-01
 * time: 1.0686421394348145
    10    -7.820996e+00     6.118728e-02
 * time: 1.0984299182891846
    11    -7.834251e+00     7.235310e-02
 * time: 1.1287169456481934
    12    -7.842009e+00     4.718987e-02
 * time: 1.1575260162353516
    13    -7.847489e+00     3.812191e-02
 * time: 1.18858194351

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846762441276                   -0.70    4.0
  2   -7.852304342708       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686622                   -2.54
  2   -7.852646686730       -9.97       -5.76
  3   -7.852646686730   +    -Inf      -11.95
  4   -7.852646686730   +    -Inf      -16.02
|ρ_newton - ρ_scf|  = 3.631061154147911e-13
|ρ_newton - ρ_scfv| = 4.006592234614938e-14
|ρ_newton - ρ_dm|   = 4.4129635351632053e-10
